In [1]:
import numpy as np
import pandas as pd 

In [2]:
dataset = {'Taste':['Salty','Spicy','Spicy','Spicy','Spicy','Sweet','Salty','Sweet','Spicy','Salty'],
       'Temperature':['Hot','Hot','Hot','Cold','Hot','Cold','Cold','Hot','Cold','Hot'],
       'Texture':['Soft','Soft','Hard','Hard','Hard','Soft','Soft','Soft','Soft','Hard'],
       'Eat':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}
#dataset={'Taste':['Salty','Spicy','Spicy','Spicy','Spicy','Sweet','Salty','Sweet','Spicy','Salty'],
#         'Eat':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}

In [3]:
df=pd.DataFrame(dataset)
df

,Taste,Temperature,Texture,Eat
0,Salty,Hot,Soft,No
1,Spicy,Hot,Soft,No
2,Spicy,Hot,Hard,Yes
3,Spicy,Cold,Hard,No
4,Spicy,Hot,Hard,Yes
5,Sweet,Cold,Soft,Yes
6,Salty,Cold,Soft,No
7,Sweet,Hot,Soft,Yes
8,Spicy,Cold,Soft,Yes
9,Salty,Hot,Hard,Yes


In [4]:
#x=df[['Taste','Temperature','Texture']]
y=df['Eat']

In [5]:
# c  (list) each element counts no. of each category in a feature
def Entropy (c):
    s,n=0,sum(c) # n total number of elements in a feature
    for i in c:
        s+=(-i/n)*np.log2(i/n) # s summation of the entropy
    return s    

In [6]:
# test Cell
Entropy([6,4])

0.9709505944546686

In [7]:
# cy (list of lists)  of summation of each class of which corespond one category of feature 
# target (list) each element counts no. of each category in the target (y) attribute
def IG (c,cy,target):
    s,n,counter=0,sum(c),0
    for i in c:
        s+=(i/n)*Entropy(cy[counter])
        counter+=1
    iG=Entropy(target)-s    
    return iG    

In [8]:
#test cell
a=[6,4] # target
b=[[1,2],[3,2],[2]] # category in a feature relative to the target
c=[3,5,2] # feature categories count
IG(c,b,a)

0.2099865470109874

In [9]:
# test cell to find the frequency of accurence of category in a column of data frame
list(y.value_counts())

[6, 4]

In [10]:
#list(x['Taste'].value_counts())

In [11]:
#list(x.columns)

In [12]:
# x is Data frame of the variables(features)
def features_categories_count (x):
    '''Returns list of lists of the counts of each feature categories'''
    features_categories_count=[]
    for i in list(x.columns):
        feature_categories_count=list(x[i].value_counts())
        features_categories_count.append(feature_categories_count)
    return features_categories_count     
        
        

In [13]:
# test cell for the function
features_categories_count(df)

[[5, 3, 2], [6, 4], [6, 4], [6, 4]]

In [14]:
#list(x['Taste'].value_counts().keys())

In [15]:
#df[df['Taste'].str.match(str('Salty'))]['Eat']

In [16]:
np.argmax(c)

1

In [17]:
#df.keys()[:-1][np.argmax(c)]

In [18]:
#np.unique(df[df['Taste'].str.match(str('Salty'))]['Eat'],return_counts=True)

In [19]:
def winner(x,y,df):
    features_categories_counts = features_categories_count (x) # >> c
    Target_categories_count=features_categories_count (y)
    Target_categories_count=Target_categories_count[0] # >> a for all features
    ########### return features_categories_counts,Target_categories_count Out: ([[5, 3, 2], [6, 4], [6, 4]], [[6, 4]])
    count=0
    IGs=[]
    for i in list(x.columns):
        categories_Relative_to_target=[]
        for n in list(x[i].value_counts().keys()):
            subtable=df[df[i].str.match(str(n))]
            category_Relative_to_target =  features_categories_count(subtable[[list(y.columns)[0]]])
            categories_Relative_to_target.append(category_Relative_to_target[0])
            #print(categories_Relative_to_target)
        #print(Target_categories_count)
        #print(features_categories_counts[count])
        I=IG(features_categories_counts[count],categories_Relative_to_target,Target_categories_count)
        IGs.append(I)
        count+=1
    return  x.keys()[np.argmax(IGs)]

In [20]:
def ID3 (df,tree=None):
   
    x=list(df.columns[:-1])
    y=list(df[['Eat']].columns)
    ######################## return x,y out :(['Taste', 'Temperature', 'Texture'], ['Eat'])
    YDF=df[['Eat']]
    XDF=df.drop('Eat',axis=1)
    node= winner(XDF,YDF,df)
    ######################## return node out:'Taste'
    #Intialize the tree as an empty Dictionary
    if tree is None:
        tree={}
        tree[node]={}
        # attribute_Categories
        #list(np.unique(x['Taste'].values))
        #list(x[node].value_counts().keys())
        #np.unique(XDF[node].values)
    for value in list(XDF[node].value_counts().keys()):
        
        subtable = df[df[node].str.match(str(value))]
        clValue,counts = np.unique(subtable['Eat'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of branch
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = ID3(subtable) #Calling the function recursively 
                   
    return tree    
    

In [27]:
tree=ID3(df)

In [32]:
tree

{'Taste': {'Spicy': {'Temperature': {'Hot': {'Texture': {'Hard': 'Yes',
      'Soft': 'No'}},
    'Cold': {'Texture': {'Hard': 'No', 'Soft': 'Yes'}}}},
  'Salty': {'Texture': {'Soft': 'No', 'Hard': 'Yes'}},
  'Sweet': 'Yes'}}

In [29]:
def predict(inst,tree):

    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            print(tree)
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

In [30]:
data={'Taste':'Salty','Temperature':'Cold','Texture':'Hard'}

In [31]:
predict(data,tree)

{'Texture': {'Soft': 'No', 'Hard': 'Yes'}}


'Yes'

In [26]:
tree.keys()

dict_keys(['Taste'])